# Reason to Select Google Colab for web scraping
* It uses the resource of google
* Our Internet Data can be saved
* Our memory is free for other task
* There is noting to worry about privecy as we are using public webste to grab data
<br>
That's it 

# Web SCraping the [collegedunia.com](https://collegedunia.com/btech-colleges)


### Importing Required Library

In [1]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen, Request
import re
import pandas as pd
import time

### Data Fream for end output 

In [2]:
data = pd.DataFrame(columns=["info","course & fees", "admission2020", "reviews", "department", "cutoff"])

### Getting College Website Links

In [3]:
my_url = Request('https://collegedunia.com/btech-colleges', headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(my_url).read()
uClient = urlopen(my_url)
page_html = uClient.read()
uClient.close()
page_soup = soup(page_html, "html.parser")

containers = page_soup.findAll('a', {'class':'college_name'})
print(len(containers))
list_url = [x['href'] for x in containers]
list_url[0]

34


'https://collegedunia.com/university/25881-indian-institute-of-technology-madras-iitm-chennai'

### Function to process Pandas html table

In [4]:
def clean_panda(data):
  temp_list = []
  isfirst = True
  for i in data:
    if isfirst:
      for j in data[i]:
        temp_list.append(j)
      isfirst = False
    else:
      for j in range(len(temp_list)):
        temp_list[j] += ", " + data[i][j]

  output = ""
  for i in range(len(temp_list)):
    output += temp_list[i] + "\n"
  return output

### Main Logic to get the data

In [5]:
for url in list_url[:10]:
  try:
    try: 
      college_data = []
      my_url = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
      time.sleep(1)
      uClient = urlopen(my_url)
      page_html = uClient.read()
      uClient.close()
      page_soup = soup(page_html, "html.parser")

      #college info 
      containers = page_soup.findAll('h1', {'class':'college_name'})
      college_data.append(containers[0].text)
    except: 
      print("College : " + url)
      
    try:
      #Course and Fees
      college_data.append(clean_panda(pd.read_html(url+"/courses-fees")[0].iloc[:,:2]))  
    except: 
      print("Cource and fee : " + url)
      
    try:
      #Admission 2020
      my_url = Request(url+"/admission", headers={'User-Agent': 'Mozilla/5.0'})
      time.sleep(1)
      uClient = urlopen(my_url)
      page_html = uClient.read()
      uClient.close()
      page_soup = soup(page_html, "html.parser")
      containers = page_soup.find('div', {'class':'admission_content_wrapper automate_client_img_article'})
      ##getting the required portion
      contant = []
      for i in containers:
        contant.append(i.text)
      college_data.append(contant[1])
    except: 
      print("Admission : " + url)
      
    try:
      #Reviews
      containers = page_soup.find('span', {'class':'rating_val'})
      college_data.append(containers.text)
    except: 
      print("Review : " + url)
      
    try:
      #Department
      my_url = Request(url+"/department", headers={'User-Agent': 'Mozilla/5.0'})
      time.sleep(1)
      uClient = urlopen(my_url)
      page_html = uClient.read()
      uClient.close()
      page_soup = soup(page_html, "html.parser")
      ##getting the required portion
      containers = page_soup.findAll('a', {'class':'ga_college_v2 college_name'})
      dep_list_str = ""
      for i in containers:
        dep_list_str += i.text + ", "
      college_data.append(dep_list_str)
    except: 
      college_data.append("information not available")
      print("Department : " + url)
      
    try:
      #Cut off
      college_data.append(clean_panda(pd.read_html(url+"/cutoff")[0]))  
    except: 
      college_data.append("information not available")
      print("cut off : " + url)
      
    #append to datafreaf
    data = data.append({
      "info": college_data[0],
      "course & fees": college_data[1],
      "admission2020": college_data[2],
      "reviews":college_data[3],
      "department": college_data[4],
      "cutoff": college_data[5]
      }, ignore_index=True)

  except:
    print("Error: " + url)

cut off : https://collegedunia.com/university/25881-indian-institute-of-technology-madras-iitm-chennai
cut off : https://collegedunia.com/university/25777-chandigarh-university-cu-chandigarh
Department : https://collegedunia.com/university/25703-iit-bombay-indian-institute-of-technology-mumbai
cut off : https://collegedunia.com/university/25703-iit-bombay-indian-institute-of-technology-mumbai
Department : https://collegedunia.com/university/26007-indian-institute-of-technology-iit-kharagpur
cut off : https://collegedunia.com/university/26007-indian-institute-of-technology-iit-kharagpur
cut off : https://collegedunia.com/university/25948-indian-institute-of-technology-iit-kanpur
cut off : https://collegedunia.com/university/25992-indian-institute-of-technology-iit-roorkee
cut off : https://collegedunia.com/university/25800-birla-institute-of-technology-and-science-bits-pilani
cut off : https://collegedunia.com/university/25833-nims-university-jaipur
Department : https://collegedunia.com

In [6]:
len(data)

10

### Data Preview

In [7]:
data

,info,course & fees,admission2020,reviews,department,cutoff
0,Indian Institute of Technology Madras - [IITM]...,"B.Tech, ₹75,116 (1st Year Fees)\nB.A + M.A, ₹3...",The last date to apply for IIT JAM 2021 is 15 ...,8.7,"Department of Management Studies, IIT Madras ...",information not available
1,"Indian Institute of Technology - [IIT], New Delhi","B.Tech, ₹2.25 Lakhs (1st Year Fees)\nM.Tech, ₹...",Registrations for GATE 2021 will close on 7 Oc...,8.7,Department of Management Studies IIT Delhi - ...,"Courses, Closing Rank 2019(Round 7), Closing R..."
2,"Chandigarh University - [CU], Chandigarh","MBA, ₹1.8 Lakhs (1st Year Fees)\nBBA, ₹96,000 ...",Latest Updates: Chandigarh University has exte...,8.6,"University School of Business, Chandigarh Uni...",information not available
3,"IIT Bombay - Indian Institute of Technology, M...","B.Tech, ₹2.11 Lakhs (1st Year Fees)\nM.Tech, ₹...",Latest Update: IIT Bombay is currently invitin...,8.9,information not available,information not available
4,"Indian Institute of Technology - [IIT], Kharagpur","B.Tech, ₹82,070 (1st Year Fees)\nM.Tech, ₹2.32...","IIT Kharagpur has 19 Academic departments, 8 m...",8.5,information not available,information not available
5,"Indian Institute of Technology - [IIT], Kanpur","B.Tech, ₹2.16 Lakhs (1st Year Fees)\nM.Tech, ₹...",The last date to apply for IIT JAM 2021 is 15 ...,8.9,"Industrial and Management Engineering, IIT Ka...",information not available
6,"Indian Institute of Technology - [IIT], Roorkee","B.Tech, ₹2.22 Lakhs (1st Year Fees)\nM.Arch, ₹...",IIT Roorkee as it is commonly known at present...,8.4,"Department of Management Studies, Indian Inst...",information not available
7,Birla Institute of Technology and Science - [B...,"B.E {Hons.}, ₹4.23 Lakhs (1st Year Fees)\nMBA,...",The Birla Institute of Technology and Science ...,8.7,"Department of Management, BITS -[DOMBITS],",information not available
8,"NIMS University, Jaipur","M.Sc, ₹50,000 (1st Year Fees)\nB.Sc, ₹40,000 (...",NIMS University was established in the year 20...,7.8,"Nims University, School of Law,",information not available
9,"Indian Institute of Technology - [IIT], Guwahati","M.A, ₹30,350 (1st Year Fees)\nB.Tech, ₹2.19 La...",IIT Guwahati was established in the year 1994 ...,8.5,information not available,information not available


### Data Fream to CSV

In [8]:
data.to_csv("data.csv")